In [ ]:
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error, recall_score, mean_absolute_error, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import time

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, validation_curve, ValidationCurveDisplay, LearningCurveDisplay, cross_val_score

from deap import base, creator, tools, algorithms
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import mlrose_hiive
from mlrose_hiive import DiscreteOpt
from mlrose_hiive import fitness
from mlrose_hiive.runners import RHCRunner, SARunner, GARunner, MIMICRunner
from concurrent.futures import ThreadPoolExecutor

In [ ]:
df = pd.read_csv('diabetes.csv', encoding='utf-8')

In [ ]:
y = df['Outcome']
y.hist()

df.columns

In [ ]:
X = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']]

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


minmaxScaler = MinMaxScaler()
minmaxScaler.fit(X_train)
minmaxScaler.transform(X_train)
minmaxScaler.fit(X_test)
minmaxScaler.transform(X_test)

#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)


# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train.to_numpy())
y_train_tensor = torch.LongTensor(y_train.to_numpy())
X_test_tensor = torch.FloatTensor(X_test.to_numpy())
y_test_tensor = torch.LongTensor(y_test.to_numpy())

# Create a DataLoader for PyTorch
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [ ]:
%%time
# MLP Classifier (Neural Network)
mlp_classifier = MLPClassifier(max_iter=1000, early_stopping=True, learning_rate_init=0.001, validation_fraction=0.1, n_iter_no_change=10, random_state=42)
mlp_classifier.fit(X_train_tensor, y_train_tensor)
y_pred = mlp_classifier.predict(X_test)
print('f1 score initial', f1_score(y_test_tensor, y_pred, average='weighted'))

mlp_param_grid = {'hidden_layer_sizes': [(64, 32), (128, 64, 32)],
                  'alpha': [0.0001, 0.001, 0.01],
                 'activation': ['identity', 'logistic', 'tanh'],
                 'solver': ['sgd', 'adam'],
                 'learning_rate': ['constant', 'adaptive']}
mlp_grid_search = GridSearchCV(mlp_classifier, mlp_param_grid, cv=5, scoring='f1_weighted')
mlp_grid_search.fit(X_train_tensor, y_train_tensor)
mlpgs = mlp_grid_search.best_estimator_.fit(X_train_tensor, y_train_tensor)
mlp_cross_val_scores = cross_val_score(mlpgs, X=X_train, y=y_train, cv=5, scoring='f1_weighted')

print(mlp_cross_val_scores) # print accuracy for each fold
print(mlp_cross_val_scores.mean()) # print mean of accuracies for all folds
print(mlp_cross_val_scores.std()) # print SD for mean of all accuracies
print('\n')

y_pred = mlpgs.predict(X_test)
print('f1 score final', f1_score(y_test_tensor, y_pred, average='weighted')) 

In [ ]:
y_pred_train = mlpgs.predict(X_train)
print(classification_report(y_train, y_pred_train))

y_pred = mlpgs.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Learning Curve
train_sizes, train_scores, test_scores = learning_curve(
    mlpgs, X_train, y_train, cv=20, scoring='f1_weighted', shuffle=True)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Train')
plt.plot(train_sizes, np.mean(test_scores, axis=1), label='Test')
plt.xlabel('Training Size')
plt.ylabel('F1')
plt.legend()
plt.title('Neural Network Learning Curve')
plt.show()

# Validation Curve
param_range = [(50,), (100,), (50, 100), (100,100), (150), (200)]  # Vary the number of hidden neurons
train_scores, test_scores = validation_curve(
    mlpgs, X_train, y_train, param_name="hidden_layer_sizes", param_range=param_range,
    cv=10, scoring='f1_weighted')

param_range = np.linspace(50, 600, 6)
plt.figure(figsize=(8, 6))
plt.plot(param_range, np.mean(train_scores, axis=1), label='Train')
plt.plot(param_range, np.mean(test_scores, axis=1), label='Test')
plt.xlabel('Hidden Neurons')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Neural Network Validation Curve')
plt.show()

# Measure Wall Clock Time
start_time = time.time()
mlpgs.fit(X_train, y_train)
training_time = time.time() - start_time

start_time = time.time()
y_pred = mlpgs.predict(X_test)
testing_time = time.time() - start_time

accuracy = f1_score(y_test, y_pred, average='weighted')

print(f"F1: {accuracy}")
print(f"Training Time: {training_time} seconds")
print(f"Testing Time: {testing_time:.3f} seconds")

In [ ]:
mlp_grid_search.best_estimator_

In [ ]:
loss_curve = mlp_grid_search.best_estimator_.loss_curve_

# Plot the loss curve
plt.figure(figsize=(7, 6))
plt.plot(loss_curve, marker='o', color='black')
plt.title('Loss Curve during Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

val_curve = mlp_grid_search.best_estimator_.validation_scores_
plt.figure(figsize=(7, 6))
plt.plot(val_curve, marker='o', color='green')
plt.title('Validation Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.show()

In [ ]:
iteration_list = [10000]
seed = 24
max_attempts = 100

In [ ]:
%%time
f1 = partial(f1_score, average="weighted")
grid_search_parameters = ({
  'learning_rate': [0.01, 0.001],
  'restarts': [5, 10, 25, 50],
  'activation': [mlrose_hiive.neural.activation.tanh] #tanh was best activation function for MLP Classifier
}) 

rhc_nn = mlrose_hiive.NeuralNetwork(hidden_nodes = [128, 64, 32], activation = 'tanh',
                                 algorithm = 'random_hill_climb', 
                                 max_iters = iteration_list[0],
                                 bias = True, 
                                is_classifier = True, 
                                 learning_rate = 0.1,
                                 early_stopping = True,
                                 max_attempts = max_attempts,
                                 random_state = seed,
                                 pop_size=200,
                                 mutation_prob=0.1,
                                   curve=True)
result = rhc_nn.fit(X_train_tensor, y_train_tensor)

In [ ]:
%%time
grid_search = {
    'max_iters': [100, 500, 1000],  # List of values to search for the 'max_iters' hyperparameter
    'hidden_layers': [[10], [20], [30, 10], [50, 30, 10]],  # List of values for 'hidden_layers'
    'learning_rate': [0.1, 0.01, 0.001],  # Values for 'learning_rate'
    'pop_size': [100, 200, 300],  # Values for 'pop_size'
    'mutation_prob': [0.1, 0.2, 0.3],  # Values for 'mutation_prob',
    "activation": [mlrose_hiive.neural.activation.tanh]
}

# run_stats, curves, cv_results, best_est = rhc_nn.run()
random_state = 24
rhc_nn = mlrose_hiive.NNGSRunner(x_train=X_train, y_train=y_train,
                           x_test=X_test, y_test=y_test,
                           algorithm=mlrose_hiive.algorithms.random_hill_climb,
                           grid_search_parameters=grid_search,
                           iteration_list = [1, 10, 50, 100, 250, 500, 1000, 2500, 5000, 10000],
                           hidden_layer_sizes=[[128, 64, 32]],
                           bias=True,
                           early_stopping=True,
                           clip_max=1,
                           max_attempts=100,
                           generate_curves=True,
                           seed=random_state,
                           n_jobs=-1
                          )
run_stats, curves, cv_results, best_est = rhc_nn.run()

In [ ]:
best_est

In [ ]:
y_pred_train = rhc_nn.predict(X_train)
print(classification_report(y_train, y_pred_train))

y_pred = rhc_nn.predict(X_test)
print(classification_report(y_test_tensor, y_pred))

In [ ]:
%%time
sa_nn = mlrose_hiive.NeuralNetwork(hidden_nodes = [128, 64, 32], activation = 'tanh',
                                 algorithm = 'simulated_annealing', 
                                 max_iters = iteration_list[0],
                                 bias = True, 
                                is_classifier = True, 
                                 learning_rate = 0.1,
                                 early_stopping = True,
                                 max_attempts = max_attempts,
                                 random_state = seed,
                                 pop_size=200,
                                 mutation_prob=0.1,
                                   curve=True)
result = sa_nn.fit(X_train_tensor, y_train_tensor)

In [ ]:
%%time
random_state = 24
grid_search = {
    "max_iters": [2500, 5000, 10000, 25000],
    "learning_rate_init": [0.001, 0.1, 0.5, 1],
    "hidden_layers_sizes": [[128, 64, 32]],
    "activation": [mlrose_hiive.neural.activation.tanh],
    "is_classifier": [True],
    "schedule": [mlrose_hiive.GeomDecay(1), mlrose_hiive.GeomDecay(10), mlrose_hiive.GeomDecay(100),
                 mlrose_hiive.GeomDecay(5), mlrose_hiive.GeomDecay(50), mlrose_hiive.GeomDecay(500),
                 mlrose_hiive.ExpDecay(1), mlrose_hiive.ExpDecay(10), mlrose_hiive.ExpDecay(100),
                 mlrose_hiive.ExpDecay(5), mlrose_hiive.ExpDecay(50), mlrose_hiive.ExpDecay(500)]
}
sa_nn = mlrose_hiive.NNGSRunner(x_train=X_train, y_train=y_train,
                           x_test=X_test, y_test=y_test,
                           algorithm=mlrose_hiive.algorithms.sa.simulated_annealing,
                           grid_search_parameters=grid_search,
                           iteration_list = [1, 10, 50, 100, 250, 500, 1000, 2500, 5000, 10000],
                           hidden_layer_sizes=[[128, 64, 32]],
                           bias=True,
                           early_stopping=True,
                           clip_max=1,
                           max_attempts=1000,
                           generate_curves=True,
                           seed=random_state,
                           n_jobs=-1
                          )
run_stats, curves, cv_results, best_est = sa_nn.run()

In [ ]:
best_est.best_params_

In [ ]:
curves

In [ ]:
y_pred = sa_nn.predict(X_train)
print(classification_report(y_train_tensor, y_pred))

y_pred = sa_nn.predict(X_test)
print(classification_report(y_test_tensor, y_pred))

In [ ]:
%%time
ga_nn = mlrose_hiive.NeuralNetwork(hidden_nodes = [128, 64, 32], activation = 'tanh',
                                 algorithm = 'genetic_alg', 
                                 max_iters = iteration_list[0],
                                 bias = True, 
                                is_classifier = True, 
                                 learning_rate = 0.1,
                                 early_stopping = True,
                                 max_attempts = max_attempts,
                                 random_state = seed,
                                 pop_size=200,
                                 mutation_prob=0.1,
                                   curve=True)
result = ga_nn.fit(X_train_tensor, y_train_tensor)

In [ ]:
%%time
grid_search = {
    "max_iters": [100, 500, 1000, 2500],
    "learning_rate_init": [0.001, 0.1, 0.5],
    "hidden_layers_sizes": [[6, 6]],
    "activation": [mlrose_hiive.neural.activation.tanh],
    "is_classifier": [True],
    "mutation_prob": [0.1, 0.25, 0.5, 0.7],
    "pop_size": [100, 200, 500, 750, 1000]    
}

ga_nn = mlrose_hiive.NNGSRunner(x_train=X_train, y_train=y_train,
                           x_test=X_test, y_test=y_test,
                           experiment_name="full_grid_search",
                           algorithm=mlrose_hiive.algorithms.genetic_alg,
                           grid_search_parameters=grid_search,
                           iteration_list = [1, 10, 50, 100, 250, 500, 1000, 2500, 5000, 10000],
                           hidden_layer_sizes=[[128, 64, 32]],
                           bias=True,
                           early_stopping=True,
                           clip_max=1,
                           max_attempts=1000,
                           generate_curves=True,
                           seed=random_state,
                           n_jobs=-1
                          )
run_stats, curves, cv_results, best_est = ga_nn.run()

In [ ]:
best_est

In [ ]:
y_pred_train = ga_nn.predict(X_train)
print(classification_report(y_train, y_pred_train))

y_pred = ga_nn.predict(X_test)
print(classification_report(y_test_tensor, y_pred))